In [1]:
import imp
import os
import cv2
import pdb
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
# from torch.nn.functional import InterpolationMode
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import logging 
import datetime
import sys
import cv2
import matplotlib.pyplot as plt
import PIL

In [2]:
gloss_df_path = "data_validation/processed_gloss.csv"
gloss_df = pd.read_csv(gloss_df_path)
gloss_df.dropna(inplace=True)
gloss_df.replace(to_replace="ASHAG", value="AŞAĞI", inplace=True)
gloss_df['glossRange'] = gloss_df['glossEnd'] - gloss_df['glossStart']
# gloss_df.head()

In [3]:
num_classes = len(gloss_df.gloss.unique())

In [4]:
import torchviz
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from torchvision import transforms

In [5]:
from pytorchvideo.data import labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomAdjustSharpness,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [6]:
video_transform = Compose([
    ApplyTransformToKey(key="video",
    transform=Compose([
        UniformTemporalSubsample(25),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        RandomShortSideScale(min_size=224, max_size=256),
        CenterCropVideo(224),
    ]),
    ),
])

In [7]:
train_path = "/home/toghrul/SLR/data/train"
val_path = "/home/toghrul/SLR/data/val"
test_path = "/home/toghrul/SLR/data/test"

In [8]:
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.metrics import classification_report
import torchmetrics

In [9]:
video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [10]:
# a = torch.randn(size=(8, num_classes)).requires_grad_()
# b = torch.randint(5, size=(8, ), dtype=torch.long)
# a, b

In [11]:
# b = F.one_hot(b, num_classes=num_classes).float()

In [12]:
# criterion = nn.BCEWithLogitsLoss()
# criterion(a, b)

In [13]:
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)
# output

In [14]:
# input, target

In [15]:
class VideoModel(LightningModule):
    def __init__(self, ):
        super(VideoModel, self).__init__()

        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(400, num_classes)

        self.lr = 1e-1
        self.batch_size = 8
        self.num_worker = 4
        self.num_steps_train = 0
        self.num_steps_val = 0

        self.metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_classes)

        #loss
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr = self.lr)
        scheduler = ReduceLROnPlateau(opt, mode="max", factor=0.05, patience=2, min_lr=1e-6)
        # scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt,
                'lr_scheduler': scheduler, 
                "monitor": "val_loss"}

    def train_dataloader(self):
        dataset = labeled_video_dataset(train_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f">>> Training step No.{self.num_steps_train}:")
        # print("Pred:", out)
        # print("GT:", label)
        # print(f"Pred:\n{out}")
        # print(f"Pred shape:\n{out.shape}")
        # print(f"Label:\n{label}")
        # print(f"Label shape:\n{label.shape}")
        # print(">>> INFO: Computing Training Loss")
        loss = self.criterion(out, label)
        print(f"Loss: {loss}")
        self.num_steps_train += 1
        # print(">>> INFO: Training Loss Computed")
        # print(">>> INFO: Computing Training Metric")
        metric = self.metric(out, label)
        print(f"Accuracy: {metric}")

        values = {"loss": loss,
                "metric": metric.detach()}
        
        self.log_dict({"step_loss": loss,
                        "step_metric": metric.detach()})
        
        return values
        
        # return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        
        self.log('training_loss', loss)
        print(f">>> Epoch end loss: {loss}")
        self.log('training_metric', metric)
        

    def val_dataloader(self):
        dataset = labeled_video_dataset(val_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        # print(">>> INFO: Computing Val Loss")
        print(f">>> Validation step No.{self.num_steps_val}:")
        loss = self.criterion(out, label)
        print(f"Loss: {loss}")
        self.num_steps_val += 1
        # print(">>> INFO: Val Loss Computed")
        # print(">>> INFO: Computing Val Metric")
        metric = self.metric(out, label)
        print(f"Accuracy: {metric}")
        

        return {"loss": loss,
                "metric": metric.detach()}
        
        # return {"loss": loss}

    def validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = labeled_video_dataset(test_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        label = F.one_hot(label, num_classes=num_classes).float()
        out = self.forward(video)
        metric = self.metric(out, label)

        return {"label": label,
                "pred": out.detach(),}

    def test_epoch_end(self, outputs):
        label=torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        # self.log('test_loss', loss)
        # self.log('test_metric', metric)
        class_labels = label.argmax(axis=1)
        class_pred = pred.argmax(axis=1)
        print(f">> Label: {class_labels}\nPred: {class_pred}")
        
        return {"prediction": class_pred,
                "labels": class_labels}




In [16]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath="checkpoints", 
                                    verbose=True, save_last=True, save_top_k=2)
lr_monitor = LearningRateMonitor(logging_interval="epoch")


In [17]:
model = VideoModel()
seed_everything(0)

trainer = Trainer(max_epochs=15,
                accelerator="gpu", devices=-1,
                precision=16,
                # accumulate_grad_batches=2,
                enable_progress_bar=True,
                # num_sanity_val_steps=0,
                callbacks=[lr_monitor, checkpoint_callback],
                log_every_n_steps=5,
                limit_train_batches=25,
                limit_val_batches=10,
                limit_test_batches=10,)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
# trainer.fit(model, ckpt_path="/home/toghrul/SLR/sign-lang/checkpoints/last.ckpt")
trainer.fit(model)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/toghrul/SLR/sign-lang/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | fc          | Linear             | 93.8 K
3 | metric      | MultilabelAccuracy | 0     
4 | criterion   | BCEWithLogitsLoss  | 0     
---------------------------------------------------
3.9 M     Trainable params
0         Non-trainable params
3.9 M     Total params
7.776     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

>>> Validation step No.0:
Loss: 0.780345618724823
Accuracy: 0.4967949390411377
>>> Validation step No.1:
Loss: 0.7818716168403625
Accuracy: 0.4978632926940918


Training: 0it [00:00, ?it/s]

>>> Training step No.0:
Loss: 0.7260846495628357
Accuracy: 0.5080128312110901
>>> Training step No.1:
Loss: 17.298078536987305
Accuracy: 0.9957265257835388
>>> Training step No.2:
Loss: 15.163996696472168
Accuracy: 0.9957265257835388
>>> Training step No.3:
Loss: 16.778846740722656
Accuracy: 0.995726466178894
>>> Training step No.4:
Loss: 17.770299911499023
Accuracy: 0.9957265257835388
>>> Training step No.5:
Loss: 18.096155166625977
Accuracy: 0.995726466178894
>>> Training step No.6:
Loss: 0.20587994158267975
Accuracy: 0.9396368265151978
>>> Training step No.7:
Loss: 0.18232017755508423
Accuracy: 0.9957265257835388
>>> Training step No.8:
Loss: 0.05963601544499397
Accuracy: 0.9941239953041077
>>> Training step No.9:
Loss: 0.5338335037231445
Accuracy: 0.9957265853881836
>>> Training step No.10:
Loss: 0.07731971144676208
Accuracy: 0.9957265257835388
>>> Training step No.11:
Loss: 0.1022660955786705
Accuracy: 0.9957265853881836
>>> Training step No.12:
Loss: 0.14754699170589447
Accuracy:

...

>>> Validation step No.140:
Loss: nan
Accuracy: 0.9957265853881836
>>> Validation step No.141:
Loss: 0.03239017724990845
Accuracy: 0.9957265257835388
>>> Epoch end loss: 0.029999999329447746
>>> Training step No.350:
Loss: 0.03921698406338692
Accuracy: 0.9957265257835388
>>> Training step No.351:
Loss: 0.03660821542143822
Accuracy: 0.9957265257835388
>>> Training step No.352:
Loss: 0.029103407636284828
Accuracy: 0.9930555820465088
>>> Training step No.353:
Loss: 0.03924057260155678
Accuracy: 0.9925214052200317
>>> Training step No.354:
Loss: 0.037757888436317444
Accuracy: 0.9957265257835388
>>> Training step No.355:
Loss: 0.031711503863334656
Accuracy: 0.9925214052200317
>>> Training step No.356:
Loss: 0.0385625846683979
Accuracy: 0.9957265257835388
>>> Training step No.357:
Loss: 0.04241446033120155
Accuracy: 0.9957265853881836
>>> Training step No.358:
Loss: 0.03247305378317833
Accuracy: 0.995726466178894
>>> Training step No.359:
Loss: 0.038072794675827026
Accuracy: 0.99252140522003

Validation: 0it [00:00, ?it/s]

>>> Validation step No.142:
Loss: nan
Accuracy: 0.9957265257835388
>>> Validation step No.143:
Loss: 0.02962273173034191
Accuracy: 0.9957265257835388
>>> Validation step No.144:
Loss: nan
Accuracy: 0.9957265257835388
>>> Validation step No.145:
Loss: 0.03904840722680092
Accuracy: 0.9957265257835388
>>> Validation step No.146:
Loss: 0.02990005351603031
Accuracy: 0.9957265257835388
>>> Validation step No.147:
Loss: 0.030375981703400612
Accuracy: 0.9957265853881836
>>> Validation step No.148:
Loss: 0.030703479424118996
Accuracy: 0.9957265257835388
>>> Validation step No.149:
Loss: nan
Accuracy: 0.9957265853881836
>>> Validation step No.150:
Loss: nan
Accuracy: 0.995726466178894


Epoch 14, global step 375: 'val_loss' was not in top 2
`Trainer.fit` stopped: `max_epochs=15` reached.


>>> Validation step No.151:
Loss: 0.03190481662750244
Accuracy: 0.995726466178894
>>> Epoch end loss: 0.03999999910593033


In [19]:
model = VideoModel.load_from_checkpoint(
    checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/last.ckpt",
    hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_38/hparams.yaml",
    map_location=None,
)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [20]:
# test_res = trainer.test(model)

In [21]:
# val_res = trainer.validate(model)

In [21]:
a = torch.randn(8, 1)

a

(tensor([[-1.1373],
         [ 1.2810],
         [ 0.6021],
         [-0.2140],
         [-0.1674],
         [-0.0197],
         [ 0.5376],
         [-0.6672]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0]))

In [33]:
a = np.random.randn(8, 4)
a, a.argmax(axis=1)

(array([[-0.88778575, -1.98079647, -0.34791215,  0.15634897],
        [ 1.23029068,  1.20237985, -0.38732682, -0.30230275],
        [-1.04855297, -1.42001794, -1.70627019,  1.9507754 ],
        [-0.50965218, -0.4380743 , -1.25279536,  0.77749036],
        [-1.61389785, -0.21274028, -0.89546656,  0.3869025 ],
        [-0.51080514, -1.18063218, -0.02818223,  0.42833187],
        [ 0.06651722,  0.3024719 , -0.63432209, -0.36274117],
        [-0.67246045, -0.35955316, -0.81314628, -1.7262826 ]]),
 array([3, 0, 3, 3, 3, 3, 1, 1]))